# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import requests
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2148458717930,
 'week52high': 136.77,
 'week52low': 55.95,
 'week52change': 0.87205458746163,
 'sharesOutstanding': 17543883824,
 'float': 0,
 'avg10Volume': 106520174,
 'avg30Volume': 99977001,
 'day200MovingAvg': 111.1,
 'day50MovingAvg': 121.37,
 'employees': 0,
 'ttmEPS': 3.28,
 'ttmDividendRate': 0.834081658638733,
 'dividendYield': 0.006556345575763212,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-26',
 'nextEarningsDate': '0',
 'peRatio': 37.11290746942731,
 'beta': 1.1507453145010056,
 'maxChangePercent': 46.90265504200467,
 'year5ChangePercent': 3.627056794316015,
 'year2ChangePercent': 2.0874274973887754,
 'year1ChangePercent': 0.845218609430636,
 'ytdChangePercent': 0.7237589982810706,
 'month6ChangePercent': 0.515276224039316,
 'month3ChangePercent': 0.025462940989920062,
 'month1ChangePercent': 0.04345705668194932,
 'day30ChangePercent': 0.04318028732001234,
 'day5ChangePercent': 0.04014803014142255}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.845218609430636

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,
                                                            data[symbol]['quote']['latestPrice'],
                                                            data[symbol]['stats']['year1ChangePercent'],
                                                            'NA'], 
                                                           index=my_columns), 
                                                ignore_index=True)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,116.80,0.409322,NA
1,AAL,17.72,-0.394983,NA
2,AAP,159.57,0.000521531,NA
3,AAPL,126.76,0.860254,NA
4,ABBV,106.70,0.290685,NA
...,...,...,...,...
500,YUM,107.96,0.0887144,NA
501,ZBH,151.83,-0.0121567,NA
502,ZBRA,376.79,0.468414,NA
503,ZION,43.09,-0.105872,NA


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe.iloc[:51, :]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,36.96,2.16363,NA
1,345,NVDA,556.96,1.59353,NA
2,31,AMD,98.44,1.42601,NA
3,275,LB,40.24,1.30233,NA
4,23,ALB,143.00,1.27208,NA
5,179,FCX,25.74,1.14112,NA
6,387,PYPL,226.68,1.12225,NA
7,339,NOW,558.31,0.997559,NA
8,388,QCOM,159.29,0.953961,NA
9,288,LRCX,510.18,0.947176,NA


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Please enter an integer')
        portfolio_size = input('Enter the size of portfolio')
        float(portfolio_size)

In [10]:
portfolio_input()

Enter the size of portfolio:100000000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe)

In [12]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
# final_dataframe['Price'].map(lambda x: math.floor(position_size/x)) == final_dataframe['Number of Shares to Buy']

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [14]:
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 
               'One-Year Price Return', 'One-Year Return Percentile', 
               'Six-Month Price Return', 'Six-Month Return Percentile', 
               'Three-Month Price Return', 'Three-Month Return Percentile', 
               'One-Month Price Return', 'One-Month Return Percentile', 'HQM Score']

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.90,N/A,0.422622,N/A,0.276201,N/A,0.189881,N/A,0.0166145,N/A,N/A
1,AAL,17.45,N/A,-0.395383,N/A,-0.076678,N/A,0.265706,N/A,0.508071,N/A,N/A
2,AAP,160.71,N/A,0.000513245,N/A,0.0577736,N/A,0.00120623,N/A,-0.0175054,N/A,N/A
3,AAPL,128.53,N/A,0.885133,N/A,0.5051,N/A,0.0249307,N/A,0.0426486,N/A,N/A
4,ABBV,110.56,N/A,0.287323,N/A,0.158141,N/A,0.17445,N/A,0.140949,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [15]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

In [16]:
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

In [17]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.90,N/A,0.422622,0.859406,0.276201,0.780198,0.189881,0.651485,0.0166145,0.291089,N/A
1,AAL,17.45,N/A,-0.395383,0.0217822,-0.076678,0.0732673,0.265706,0.827723,0.508071,0.964356,N/A
2,AAP,160.71,N/A,0.000513245,0.354455,0.0577736,0.316832,0.00120623,0.154455,-0.0175054,0.170297,N/A
3,AAPL,128.53,N/A,0.885133,0.976238,0.5051,0.948515,0.0249307,0.239604,0.0426486,0.358416,N/A
4,ABBV,110.56,N/A,0.287323,0.762376,0.158141,0.550495,0.17445,0.615842,0.140949,0.659406,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.92,N/A,0.0879306,0.50099,0.102856,0.427723,0.111229,0.441584,0.0641516,0.451485,N/A
501,ZBH,146.48,N/A,-0.0127312,0.328713,0.0639539,0.334653,0.0441548,0.29505,0.0516961,0.40396,N/A
502,ZBRA,386.82,N/A,0.469216,0.883168,0.349027,0.871287,0.449323,0.950495,0.0812268,0.49505,N/A
503,ZION,43.93,N/A,-0.104679,0.194059,0.135492,0.508911,0.297869,0.865347,0.363555,0.918812,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [18]:
hqm_dataframe['HQM Score'] = hqm_dataframe.apply(lambda row: row[[f'{time_period} Return Percentile' for time_period in time_periods]].mean(), axis=1)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [19]:
hqm_dataframe = hqm_dataframe.sort_values('HQM Score', ascending=False)[:51]

In [20]:
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.1300,N/A,1.13168,0.990099,1.28056,1,0.557611,0.978218,0.287658,0.865347,0.958416
1,ALB,145.0000,N/A,1.25444,0.992079,0.686412,0.978218,0.49136,0.968317,0.247705,0.829703,0.942079
2,LRCX,509.6900,N/A,0.935394,0.980198,0.688652,0.980198,0.533917,0.972277,0.249112,0.831683,0.941089
3,URI,247.7400,N/A,0.581199,0.938614,0.539434,0.966337,0.399656,0.936634,0.300397,0.881188,0.930693
4,AMAT,89.7000,N/A,0.620677,0.946535,0.507044,0.950495,0.487797,0.964356,0.270915,0.847525,0.927228
5,MU,73.0800,N/A,0.529481,0.916832,0.366173,0.885149,0.572543,0.980198,0.324291,0.89505,0.919307
6,LB,42.2100,N/A,1.33913,0.994059,1.18727,0.99802,0.336903,0.90099,0.210479,0.774257,0.916832
7,KLAC,276.5000,N/A,0.679243,0.956436,0.39976,0.910891,0.401242,0.938614,0.161508,0.706931,0.878218
8,XLNX,150.3900,N/A,0.663937,0.952475,0.563211,0.970297,0.466085,0.958416,0.112482,0.59802,0.869802
9,QRVO,173.1000,N/A,0.570885,0.934653,0.45751,0.934653,0.423138,0.946535,0.138784,0.655446,0.867822


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the size of portfolio:2000000000


In [22]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [26]:
writer.save()